# Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import SparkContext, SQLContext
spark = SparkSession \
    .builder \
    .appName("Sprint 5 - Semana 1") \
    .getOrCreate()
SQLContext = SQLContext(spark.sparkContext)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/18 14:15:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/18 14:15:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/home/guilherme.cunha/.local/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Exercicíos

## Exercicio 1

Crie um array de 250 itens e aplique o método reverse para inverter o conteúdo.

### Forma 1 (convencional)

Criando dataframe

In [2]:
ex = spark.range(0,250)
ex.count()

250

Verificando

In [3]:
ex.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+
only showing top 20 rows



Utilizando o método reverse

In [4]:
ex.select(reverse(ex.id).alias('id reverso')).show()

+----------+
|id reverso|
+----------+
|         0|
|         1|
|         2|
|         3|
|         4|
|         5|
|         6|
|         7|
|         8|
|         9|
|        01|
|        11|
|        21|
|        31|
|        41|
|        51|
|        61|
|        71|
|        81|
|        91|
+----------+
only showing top 20 rows



### Forma 2 (utilizando UDFs)

Criando UDF

In [5]:
@udf('int')
def reverser(numero: int):
    result = ''
    numero = str(numero)
    result = numero[::-1]
    return int(result)

Criando coluna através da UDF

In [6]:
ex_reverso = ex.withColumn('id reverso', reverser('id')) 

Verificando

In [7]:
ex_reverso.show()

+---+----------+
| id|id reverso|
+---+----------+
|  0|         0|
|  1|         1|
|  2|         2|
|  3|         3|
|  4|         4|
|  5|         5|
|  6|         6|
|  7|         7|
|  8|         8|
|  9|         9|
| 10|         1|
| 11|        11|
| 12|        21|
| 13|        31|
| 14|        41|
| 15|        51|
| 16|        61|
| 17|        71|
| 18|        81|
| 19|        91|
+---+----------+
only showing top 20 rows



## Exercicio 2

Crie uma lista de 20 animais, ordene-os e itere para imprimi-las individualmente cada um deles, depois salve num arquivo texto emformato CSV

Criando lista

In [8]:
list = ['Macaco', 'Arara', 'Cachorro', 'Gato', 'Tamandua', 
        'Cobra', 'Boi', 'Cavalo', 'Rato', 'Golfinho', 
        'Leão', 'Onça', 'Elefante', 'Girafa', 'Raposa', 
        'Pinguim', 'Jacaré', 'Panda', 'Tartaruga', 'Porco']

Criando RDD

In [9]:
animais = spark.sparkContext.parallelize(list)

Ordenando RDD

In [10]:
animais.sortBy(lambda x : x).collect()

['Arara',
 'Boi',
 'Cachorro',
 'Cavalo',
 'Cobra',
 'Elefante',
 'Gato',
 'Girafa',
 'Golfinho',
 'Jacaré',
 'Leão',
 'Macaco',
 'Onça',
 'Panda',
 'Pinguim',
 'Porco',
 'Raposa',
 'Rato',
 'Tamandua',
 'Tartaruga']

Printando RDD

In [11]:
for x in animais.sortBy(lambda x : x).collect():
    print(x)

Arara
Boi
Cachorro
Cavalo
Cobra
Elefante
Gato
Girafa
Golfinho
Jacaré
Leão
Macaco
Onça
Panda
Pinguim
Porco
Raposa
Rato
Tamandua
Tartaruga


Exportando para CSV

In [12]:
row = Row("animal")
animais.map(row).toDF().write.csv('animais.csv', sep=';', header=True)

# Lab 3



## Passo a passo

In [13]:
df_nomes = spark.read.csv("nomes.csv", header=True, inferSchema=True)

In [14]:
type(df_nomes)

pyspark.sql.dataframe.DataFrame

In [15]:
df_nomes.show(5)

+--------+----+-----+----+
|    nome|sexo|total| ano|
+--------+----+-----+----+
|Jennifer|   F|54336|1983|
| Jessica|   F|45278|1983|
|  Amanda|   F|33752|1983|
|  Ashley|   F|33292|1983|
|   Sarah|   F|27228|1983|
+--------+----+-----+----+
only showing top 5 rows



In [16]:
df_nomes.printSchema()

root
 |-- nome: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- total: integer (nullable = true)
 |-- ano: integer (nullable = true)



In [17]:
df_nomes.select(df_nomes.nome, df_nomes.sexo).show(5)

+--------+----+
|    nome|sexo|
+--------+----+
|Jennifer|   F|
| Jessica|   F|
|  Amanda|   F|
|  Ashley|   F|
|   Sarah|   F|
+--------+----+
only showing top 5 rows



In [18]:
df_select = df_nomes.select(df_nomes.nome,
    df_nomes.ano,
    df_nomes.ano > 1950,
    df_nomes.ano + 501
)
df_select.show(5)

+--------+----+------------+-----------+
|    nome| ano|(ano > 1950)|(ano + 501)|
+--------+----+------------+-----------+
|Jennifer|1983|        true|       2484|
| Jessica|1983|        true|       2484|
|  Amanda|1983|        true|       2484|
|  Ashley|1983|        true|       2484|
|   Sarah|1983|        true|       2484|
+--------+----+------------+-----------+
only showing top 5 rows



In [19]:
df_teste = df_nomes.withColumn('nome', df_nomes.nome).withColumn('ano ++', df_nomes.ano + 501)
    
df_teste .show(5)

+--------+----+-----+----+------+
|    nome|sexo|total| ano|ano ++|
+--------+----+-----+----+------+
|Jennifer|   F|54336|1983|  2484|
| Jessica|   F|45278|1983|  2484|
|  Amanda|   F|33752|1983|  2484|
|  Ashley|   F|33292|1983|  2484|
|   Sarah|   F|27228|1983|  2484|
+--------+----+-----+----+------+
only showing top 5 rows



In [20]:
df_nomes.registerTempTable("pessoas")

/home/guilherme.cunha/.local/lib/python3.10/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [21]:
SQLContext.sql("select count(*) from pessoas").show()

+--------+
|count(1)|
+--------+
| 1825433|
+--------+



In [22]:
SQLContext.sql("SELECT nome, ano, ano + 100 as futuro FROM pessoas").show(5)

+--------+----+------+
|    nome| ano|futuro|
+--------+----+------+
|Jennifer|1983|  2083|
| Jessica|1983|  2083|
|  Amanda|1983|  2083|
|  Ashley|1983|  2083|
|   Sarah|1983|  2083|
+--------+----+------+
only showing top 5 rows



In [23]:
df_nomes.filter(df_nomes.ano > 1990).select(df_nomes.nome, df_nomes.ano).show(5)

+--------+----+
|    nome| ano|
+--------+----+
|  Ashley|1991|
| Jessica|1991|
|Brittany|1991|
|  Amanda|1991|
|Samantha|1991|
+--------+----+
only showing top 5 rows



In [24]:
df_nomes.filter(df_nomes.ano > 1990).select(df_nomes.nome, df_nomes.ano).orderBy(df_nomes.ano, df_nomes.nome).show(5)

+-------+----+
|   nome| ano|
+-------+----+
| Aaisha|1991|
| Aakash|1991|
|Aaliyah|1991|
| Aamber|1991|
|Aaminah|1991|
+-------+----+
only showing top 5 rows



In [25]:
df_nomes.filter(df_nomes.ano > 1990).select(df_nomes.nome, df_nomes.ano).orderBy(df_nomes.ano.desc(), df_nomes.nome.desc()).show(5)

+--------+----+
|    nome| ano|
+--------+----+
|   Zyyon|2014|
|Zytaveon|2014|
| Zyshawn|2014|
|   Zyrus|2014|
|   Zyron|2014|
+--------+----+
only showing top 5 rows



In [26]:
df_nomes.select("nome", "ano", (df_nomes.ano > 2000).alias('recente')).show(5)

+--------+----+-------+
|    nome| ano|recente|
+--------+----+-------+
|Jennifer|1983|  false|
| Jessica|1983|  false|
|  Amanda|1983|  false|
|  Ashley|1983|  false|
|   Sarah|1983|  false|
+--------+----+-------+
only showing top 5 rows



In [27]:
spark.sql("select nome, ano, ano>2000 as recente FROM pessoas").show(5)

+--------+----+-------+
|    nome| ano|recente|
+--------+----+-------+
|Jennifer|1983|  false|
| Jessica|1983|  false|
|  Amanda|1983|  false|
|  Ashley|1983|  false|
|   Sarah|1983|  false|
+--------+----+-------+
only showing top 5 rows



In [28]:
df_nomes.groupBy('ano').count().show(5)

+----+-----+
| ano|count|
+----+-----+
|1990|24713|
|1988|22358|
|1994|25997|
|1991|25104|
|1989|23767|
+----+-----+
only showing top 5 rows



## Exercicios

### Exercicío 1

#### Qual o sexo que mais aparece

Agrupando e contando baseado na coluna 'sexo'

In [29]:
df_nomes.groupBy('sexo').count().show()

+----+-------+
|sexo|  count|
+----+-------+
|   F|1081683|
|   M| 743750|
+----+-------+



Utilizando a API de sql para realizar a mesma consulta acima

In [30]:
spark.sql("SELECT sexo, count(sexo) FROM pessoas GROUP BY sexo").show()

+----+-----------+
|sexo|count(sexo)|
+----+-----------+
|   F|    1081683|
|   M|     743750|
+----+-----------+



#### Qual foi a diferença entre o sexo Masculino e o Feminino

Realizando a mesma contagem do exercicio anterior, porém, salvando a tabela para utilizar futuramente sob o prefixo 'sex'

In [31]:
spark.sql("SELECT sexo, count(sexo) as soma FROM pessoas GROUP BY sexo").registerTempTable('sex')

Realizando a diferença entre os dois valores presentes na tabela criada acima

In [32]:
spark.sql('select MAX(soma)-MIN(soma) as diferenca from sex').show()

+---------+
|diferenca|
+---------+
|   337933|
+---------+



### Exercicío 2 - 
Crie uma nova coluna no DataFrame que contenha apenas números. Nesta coluna chamada RANKING deverá ter o número que representa a colocação do nome na lista, sendo o número 1 o nome mais usado em todos os anos.

Ordenando o dataframe baseado primeiramente na coluna 'total' e após baseado na coluna 'nome'

In [33]:
df_nomes = df_nomes.orderBy(col("total").desc(),col("nome").desc())

Adicionando uma nova coluna denominada 'RANKING' que contém um indice definido pelo método 'monotonically_increasing_id', o '+1' é necessário pois o método inicia a contagem no indice 0

In [34]:
df_nomes = df_nomes.select("*").withColumn("RANKING", monotonically_increasing_id()+1)

Verificando

In [35]:
df_nomes.show(20)

+-------+----+-----+----+-------+
|   nome|sexo|total| ano|RANKING|
+-------+----+-----+----+-------+
|  Linda|   F|99680|1947|      1|
|  Linda|   F|96205|1948|      2|
|  James|   M|94755|1947|      3|
|Michael|   M|92709|1957|      4|
| Robert|   M|91642|1947|      5|
|  Linda|   F|91010|1949|      6|
|Michael|   M|90633|1956|      7|
|Michael|   M|90519|1958|      8|
|  James|   M|88596|1948|      9|
|Michael|   M|88485|1954|     10|
|   John|   M|88318|1947|     11|
|Michael|   M|88283|1955|     12|
|  James|   M|87425|1946|     13|
|  James|   M|87175|1951|     14|
|  James|   M|87083|1952|     15|
|Michael|   M|86916|1961|     16|
|  James|   M|86856|1949|     17|
| Robert|   M|86572|1952|     18|
| Robert|   M|86327|1951|     19|
|  James|   M|86277|1954|     20|
+-------+----+-----+----+-------+
only showing top 20 rows



### Exercicío 3 - 
Crie 03 novas colunas no DataFrame que contenha apenas Sim e Não. Cada coluna deve-se chamar DECADA_80, DECADA_90, DECADA_00 e o conteúdo deve ser SIM ou NÃO  de acordo com o ano definido na tabela.

Definindo uma função que identifica se o primeiro ano passado corresponde a década do segundo ano passo como parametro, retorna string 'sim' ou 'nao'

In [36]:
def check_year(ano_name, ano_col):
    decada = ano_name - (ano_name%10)
    result = 'SIM' if ano_name >= ano_col and ano_name < ano_col+10 else 'NAO'
    return result

Definindo a função acima como uma UDF

In [37]:
decada = udf(check_year, StringType())

Criando três colunas e utilizando a udf 'decada' para checar se o ano corresponde a decada informada na coluna

In [38]:
df_nomes = df_nomes.select("*").withColumn("DECADA_80", decada('ano', lit(1980))).withColumn("DECADA_90", decada('ano', lit(1990))).withColumn("DECADA_00", decada('ano', lit(2000)))

Verificando

In [39]:
df_nomes.show()

+-------+----+-----+----+-------+---------+---------+---------+
|   nome|sexo|total| ano|RANKING|DECADA_80|DECADA_90|DECADA_00|
+-------+----+-----+----+-------+---------+---------+---------+
|  Linda|   F|99680|1947|      1|      NAO|      NAO|      NAO|
|  Linda|   F|96205|1948|      2|      NAO|      NAO|      NAO|
|  James|   M|94755|1947|      3|      NAO|      NAO|      NAO|
|Michael|   M|92709|1957|      4|      NAO|      NAO|      NAO|
| Robert|   M|91642|1947|      5|      NAO|      NAO|      NAO|
|  Linda|   F|91010|1949|      6|      NAO|      NAO|      NAO|
|Michael|   M|90633|1956|      7|      NAO|      NAO|      NAO|
|Michael|   M|90519|1958|      8|      NAO|      NAO|      NAO|
|  James|   M|88596|1948|      9|      NAO|      NAO|      NAO|
|Michael|   M|88485|1954|     10|      NAO|      NAO|      NAO|
|   John|   M|88318|1947|     11|      NAO|      NAO|      NAO|
|Michael|   M|88283|1955|     12|      NAO|      NAO|      NAO|
|  James|   M|87425|1946|     13|      N

### Exercicío 4 - 
Remova do Dataframe a coluna DECADA_00 e mostrar o novo Dataframe.

Removendo a coluna 'DECADA_00'

In [40]:
df_nomes = df_nomes.drop('DECADA_00')

Verificando

In [41]:
df_nomes.show()

+-------+----+-----+----+-------+---------+---------+
|   nome|sexo|total| ano|RANKING|DECADA_80|DECADA_90|
+-------+----+-----+----+-------+---------+---------+
|  Linda|   F|99680|1947|      1|      NAO|      NAO|
|  Linda|   F|96205|1948|      2|      NAO|      NAO|
|  James|   M|94755|1947|      3|      NAO|      NAO|
|Michael|   M|92709|1957|      4|      NAO|      NAO|
| Robert|   M|91642|1947|      5|      NAO|      NAO|
|  Linda|   F|91010|1949|      6|      NAO|      NAO|
|Michael|   M|90633|1956|      7|      NAO|      NAO|
|Michael|   M|90519|1958|      8|      NAO|      NAO|
|  James|   M|88596|1948|      9|      NAO|      NAO|
|Michael|   M|88485|1954|     10|      NAO|      NAO|
|   John|   M|88318|1947|     11|      NAO|      NAO|
|Michael|   M|88283|1955|     12|      NAO|      NAO|
|  James|   M|87425|1946|     13|      NAO|      NAO|
|  James|   M|87175|1951|     14|      NAO|      NAO|
|  James|   M|87083|1952|     15|      NAO|      NAO|
|Michael|   M|86916|1961|   

### Exercicío 5 - 
Gerar dois arquivos CSV. O primeiro com os nomes do sexo masculino nascidos na década de 80. O segundo com os nomes do sexo feminino nascidos na década de 80.

Criando view

In [42]:
df_nomes.createOrReplaceTempView('temp')

Criando um dataframe somente com pessoas do sexo feminino nascidas na década de 80

In [43]:
f_80 = spark.sql('select * from temp where sexo = "F" and ano between 1980 and 1989 order by RANKING asc')

Exportando para csv

In [44]:
f_80.write.csv('output',mode='append')

Criando um dataframe somente com pessoas do sexo masculino nascidas na década de 80

In [45]:
m_80 = spark.sql('select * from temp where sexo = "M" and ano between 1980 and 1989 order by RANKING asc')

Exportando para csv

In [46]:
m_80.write.csv('output',mode='append')